In [ ]:
import cv2
import torch
import numpy as np
import os
from sklearn.cluster import KMeans
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from google.colab import files
import chromadb # Yeni import
from sentence_transformers import SentenceTransformer # Yeni import

# --- AYARLAR ---
SKIP_FRAMES = 30       # Her 30 karede bir işlem yap (Yaklaşık 1 FPS)
BLUR_THRESHOLD = 100.0 # Bulanıklık eşiği (Düşük değerler daha bulanık kabul edilir)
MIN_CLUSTERS = 5       # En az kaç keyframe olsun
MAX_CLUSTERS = 20      # En fazla kaç keyframe olsun

# 1. Cihazı Belirle (Önceki hücreden alınmıştır)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 KULLANILAN CİHAZ: {device}")

# H100 vb. için Hızlandırma Ayarları
if device == "cuda":
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

# 2. Model Yükleme (DINOv2 - Large)
print("DINOv2 (Large) Modeli Yükleniyor...")
dinov2 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14')
dinov2.to(device)
dinov2.eval()

# 3. Video İşleme
video_path = "/content/video.mp4"
if not os.path.exists(video_path):
    print("Video bulunamadı, lütfen yükleyiniz:")
    uploaded = files.upload()
    # Fix: Rename the uploaded file to match video_path
    for filename in uploaded.keys():
        if filename != os.path.basename(video_path):
            os.rename(filename, video_path)
            print(f"Uploaded file '{filename}' renamed to '{video_path}'.")

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0: fps = 30 # Hata önleyici

frames = []
frame_ids = []
frame_count = 0

print("Video taranıyor (Örnekleme + Bulanıklık Kontrolü)...")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 1. Adım: Örnekleme (Sadece belirli aralıklarla kare al)
    if frame_count % SKIP_FRAMES == 0:
        # 2. Adım: Bulanıklık Kontrolü
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()

        if laplacian_var > BLUR_THRESHOLD:
            frames.append(frame)
            frame_ids.append(frame_count)

    frame_count += 1

cap.release()
print(f"Toplam Kare: {frame_count}")
print(f"Seçilen Net Kare Sayısı: {len(frames)}")

if len(frames) == 0:
    raise ValueError("Hiç uygun kare bulunamadı! Eşik değerini düşürmeyi deneyin.")

# 4. Embedding Çıkarma
embeddings_dinov2 = [] # Dinov2 embeddingleri için yeni isim
print("DINOv2 Embeddingler çıkarılıyor...")

for frame in frames:
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    # DINOv2 için resize
    image = image.resize((224, 224))
    image_tensor = torch.tensor(np.array(image)).permute(2, 0, 1).float() / 255.0
    image_tensor = image_tensor.unsqueeze(0).to(device)

    with torch.no_grad():
        embedding = dinov2(image_tensor)

    embeddings_dinov2.append(embedding.cpu().numpy()[0])

embeddings_dinov2 = np.array(embeddings_dinov2)

# 5. Dinamik Kümeleme (Keyframe Seçimi)
# Örnek: Her 10 saniye için 1 keyframe, ama min 5 max 20 arasında tut.
duration_sec = frame_count / fps
dynamic_k = int(duration_sec / 10)
NUM_CLUSTERS = max(MIN_CLUSTERS, min(dynamic_k, MAX_CLUSTERS, len(frames)))

print(f"Video Süresi: {duration_sec:.1f}sn -> Hedeflenen Keyframe Sayısı: {NUM_CLUSTERS}")

kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42, n_init=10) # n_init eklendi
labels = kmeans.fit_predict(embeddings_dinov2)
keyframe_indices = []

for cluster_id in range(NUM_CLUSTERS):
    cluster_indices = np.where(labels == cluster_id)[0]
    cluster_embeddings = embeddings_dinov2[cluster_indices]
    center = kmeans.cluster_centers_[cluster_id]
    distances = np.linalg.norm(cluster_embeddings - center, axis=1)
    best = cluster_indices[np.argmin(distances)]
    keyframe_indices.append(best)

keyframe_indices = sorted(keyframe_indices)

# 6. Captioning (BLIP-2) ve ChromaDB Entegrasyonu
print("BLIP-2 Modeli Yükleniyor...")
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
blip_model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b").to(device)

# Sentence Transformer modeli yükle (Tekrar tanımlandı, önceki adımda da vardı, güvenli olsun diye burada tekrar)
sentence_model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

# ChromaDB istemcisi ve koleksiyonu başlat (Tekrar tanımlandı, önceki adımda da vardı, güvenli olsun diye burada tekrar)
chroma_client = chromadb.Client()
keyframe_collection = chroma_client.get_or_create_collection(name="keyframe_captions")

# ChromaDB'yi temizle (Her çalıştırmada sıfırdan başlamak için)
existing_ids = keyframe_collection.get()['ids']
if existing_ids:
    keyframe_collection.delete(ids=existing_ids) # Tüm mevcut verileri sil
    print("ChromaDB koleksiyonu temizlendi.")
else:
    print("ChromaDB koleksiyonu zaten boş.")


print("\n--- Keyframe'ler İşleniyor ve ChromaDB'ye Kaydediliyor ---")
caption_data = [] # Tüm caption'ları tutmak için

for i, idx in enumerate(keyframe_indices):
    frame = frames[idx]
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    inputs = processor(image, return_tensors="pt").to(device)

    with torch.no_grad():
        output = blip_model.generate(**inputs, max_new_tokens=30)

    caption = processor.decode(output[0], skip_special_tokens=True)

    # Caption embedding'ini oluştur
    caption_embedding = sentence_model.encode(caption).tolist()

    # Metadata hazırla
    frame_id = str(frame_ids[idx])
    # FIX: Store timestamp as float instead of string
    timestamp_float = float(frame_ids[idx]/fps) # Convert to float
    timestamp_str = f"{timestamp_float:.1f}s" # For display purposes
    unique_id = f"keyframe_{i}"

    # ChromaDB'ye ekle
    keyframe_collection.add(
        embeddings=[caption_embedding],
        metadatas=[{"frame_id": frame_id, "timestamp": timestamp_float}], # Store float
        documents=[caption],
        ids=[unique_id]
    )
    caption_data.append(f"Frame {frame_id} ({timestamp_str}): {caption}")

    print(f"Frame {frame_id} ({timestamp_str}): {caption}")

print("\n✅ Keyframe'ler işlendi ve ChromaDB'ye kaydedildi.")

# 7. Örnek Anlamsal Arama Sorgusu
print("\n--- Örnek Anlamsal Arama (ChromaDB) ---")
query_text = "they are fighting"
query_embedding = sentence_model.encode(query_text).tolist()

print(f"Sorgu: '{query_text}' için arama yapılıyor...")

results = keyframe_collection.query(
    query_embeddings=[query_embedding],
    n_results=3, # En yakın 3 sonucu getir
    include=['documents', 'metadatas', 'distances']
)

print("\n--- Arama Sonuçları ---")
if results['documents']:
    for i, doc in enumerate(results['documents'][0]):
        metadata = results['metadatas'][0][i]
        distance = results['distances'][0][i]
        # FIX: Ensure timestamp is displayed correctly after float storage
        display_timestamp = f"{metadata['timestamp']:.1f}s" if isinstance(metadata['timestamp'], float) else metadata['timestamp']
        print(f"  Sonuç {i+1} (Mesafe: {distance:.2f}):")
        print(f"    Frame ID: {metadata['frame_id']}, Zaman: {display_timestamp}")
        print(f"    Açıklama: {doc}")
else:
    print("Hiç sonuç bulunamadı.")

print("✅ Anlamsal arama tamamlandı.")

print("\n--- Gelişmiş Anlamsal Arama (ChromaDB) ---")




🚀 KULLANILAN CİHAZ: cuda
DINOv2 (Large) Modeli Yükleniyor...


/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)


Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip


/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitl14/dinov2_vitl14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitl14_pretrain.pth


...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ChromaDB koleksiyonu zaten boş.

--- Keyframe'ler İşleniyor ve ChromaDB'ye Kaydediliyor ---
Frame 630 (26.3s): two people are posing for a selfie in front of a sign

Frame 930 (38.8s): the stage is lit up at night with lights

Frame 1470 (61.3s): a starbucks coffee stand in the middle of a field

Frame 1800 (75.1s): a group of men in blue uniforms standing in a room

Frame 2670 (111.4s): a man is playing basketball on a court at night

Frame 2910 (121.4s): a man and woman are taking a selfie together

Frame 3540 (147.6s): a man in a blue shirt is holding a volleyball

Frame 4500 (187.7s): a woman in a blue shirt is talking to the camera

Frame 6360 (265.3s): a shirtless man sitting on a bench in the grass

Frame 7170 (299.0s): a woman in a blue shirt sitting on a table

Frame 7380 (307.8s): a man in a blue shirt is smiling for the camera

Frame 7890 (329.1s): a man and woman in blue shirts are taking a selfie

Frame 8520 (355.4s): a man and woman eating food at a restaurant

Frame 8910

In [ ]:
!pip install chromadb

In [ ]:
# 1. Farklı bir query_text belirleyin
query_text_advanced = "they are celebrating"
print(f"Gelişmiş Sorgu: '{query_text_advanced}' için arama yapılıyor...")

# 2. Yeni query_text için embedding oluşturun
query_embedding_advanced = sentence_model.encode(query_text_advanced).tolist()

# 3. ve 4. ChromaDB'de daha fazla sonuç ve metadata filtrelemesi ile arama yapın
# Örnek: Yalnızca 100 saniyeden sonraki kareleri içeren sonuçları getir
results_advanced = keyframe_collection.query(
    query_embeddings=[query_embedding_advanced],
    n_results=5, # En yakın 5 sonucu getir
    where={
        "timestamp": {"$gt": 100.0} # FIX: Compare against float instead of string
    },
    include=['documents', 'metadatas', 'distances']
)

print("\n--- Gelişmiş Arama Sonuçları ---")
if results_advanced['documents']:
    for i, doc in enumerate(results_advanced['documents'][0]):
        metadata = results_advanced['metadatas'][0][i]
        distance = results_advanced['distances'][0][i]
        # FIX: Ensure timestamp is displayed correctly after float storage
        display_timestamp_advanced = f"{metadata['timestamp']:.1f}s" if isinstance(metadata['timestamp'], float) else metadata['timestamp']
        print(f"  Sonuç {i+1} (Mesafe: {distance:.2f}):")
        print(f"    Frame ID: {metadata['frame_id']}, Zaman: {display_timestamp_advanced}")
        print(f"    Açıklama: {doc}")
else:
    print("Hiç sonuç bulunamadı.")

print("✅ Gelişmiş anlamsal arama tamamlandı.")

Gelişmiş Sorgu: 'they are celebrating' için arama yapılıyor...

--- Gelişmiş Arama Sonuçları ---
  Sonuç 1 (Mesafe: 1.37):
    Frame ID: 10680, Zaman: 445.4s
    Açıklama: a group of people in blue shirts are standing on a stage

  Sonuç 2 (Mesafe: 1.38):
    Frame ID: 11850, Zaman: 494.2s
    Açıklama: a man and woman are smiling while standing in front of a soccer field

  Sonuç 3 (Mesafe: 1.53):
    Frame ID: 9360, Zaman: 390.4s
    Açıklama: a group of people sitting on the ground talking

  Sonuç 4 (Mesafe: 1.55):
    Frame ID: 11280, Zaman: 470.5s
    Açıklama: the big brother houseguests are doing a weightlifting competition

  Sonuç 5 (Mesafe: 1.56):
    Frame ID: 8520, Zaman: 355.4s
    Açıklama: a man and woman eating food at a restaurant

✅ Gelişmiş anlamsal arama tamamlandı.


In [ ]:
# 1. Farklı bir query_text belirleyin
query_text_advanced = "Security forces or police operations"
print(f"Gelişmiş Sorgu: '{query_text_advanced}' için arama yapılıyor...")

# 2. Yeni query_text için embedding oluşturun
query_embedding_advanced = sentence_model.encode(query_text_advanced).tolist()

# 3. ve 4. ChromaDB'de daha fazla sonuç ve metadata filtrelemesi ile arama yapın
# Örnek: Yalnızca 100 saniyeden sonraki kareleri içeren sonuçları getir
results_advanced = keyframe_collection.query(
    query_embeddings=[query_embedding_advanced],
    n_results=5, # En yakın 5 sonucu getir
    where={
        "timestamp": {"$gt": 100.0} # FIX: Compare against float instead of string
    },
    include=['documents', 'metadatas', 'distances']
)

print("\n--- Gelişmiş Arama Sonuçları ---")
if results_advanced['documents']:
    for i, doc in enumerate(results_advanced['documents'][0]):
        metadata = results_advanced['metadatas'][0][i]
        distance = results_advanced['distances'][0][i]
        # FIX: Ensure timestamp is displayed correctly after float storage
        display_timestamp_advanced = f"{metadata['timestamp']:.1f}s" if isinstance(metadata['timestamp'], float) else metadata['timestamp']
        print(f"  Sonuç {i+1} (Mesafe: {distance:.2f}):")
        print(f"    Frame ID: {metadata['frame_id']}, Zaman: {display_timestamp_advanced}")
        print(f"    Açıklama: {doc}")
else:
    print("Hiç sonuç bulunamadı.")

print("✅ Gelişmiş anlamsal arama tamamlandı.")

Gelişmiş Sorgu: 'Security forces or police operations' için arama yapılıyor...

--- Gelişmiş Arama Sonuçları ---
  Sonuç 1 (Mesafe: 1.64):
    Frame ID: 5130, Zaman: 205.2s
    Açıklama: turkish police arrest man who killed 3 people in a mall

  Sonuç 2 (Mesafe: 1.71):
    Frame ID: 5970, Zaman: 238.8s
    Açıklama: a television screen showing a television news report about the collapse of a building

  Sonuç 3 (Mesafe: 1.72):
    Frame ID: 7860, Zaman: 314.4s
    Açıklama: a television screen showing a hospital room with a bed and a chair

  Sonuç 4 (Mesafe: 1.73):
    Frame ID: 3840, Zaman: 153.6s
    Açıklama: a television screen showing a room with a lot of stuff

  Sonuç 5 (Mesafe: 1.76):
    Frame ID: 7050, Zaman: 282.0s
    Açıklama: a television screen showing a room with beds and mattresses

✅ Gelişmiş anlamsal arama tamamlandı.


In [ ]:
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt # İsteğe bağlı olarak ızgara düzeni için kullanılabilir

# Video yakalama nesnesini yeniden aç
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError("Video dosyası açılamadı!")

# Orijinal FPS'i al, zaman dönüştürme için gerekli
fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0: fps = 30 # Bulunamazsa varsayılan değer

print("\n--- ChromaDB Arama Sonuçlarının Görselleştirilmesi ---")

# Önceki hücreden gelen results_advanced'ı kullan
if results_advanced['documents']:
    for i, doc in enumerate(results_advanced['documents'][0]):
        metadata = results_advanced['metadatas'][0][i]
        frame_id = int(metadata['frame_id'])
        timestamp_float = metadata['timestamp']
        distance = results_advanced['distances'][0][i]

        # Kareye git
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        ret, frame = cap.read()

        if ret:
            # Kareyi etiketle
            caption = doc.strip()
            text_frame_id = f"Frame ID: {frame_id}"
            text_timestamp = f"Time: {timestamp_float:.1f}s"
            text_caption = f"Caption: {caption}"
            text_distance = f"Distance: {distance:.2f}"

            # Görüntülemek için metin hazırla
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.7
            font_thickness = 2
            text_color = (0, 255, 255) # Sarı
            bg_color = (0, 0, 0) # Metin için siyah arka plan

            y0, dy = 30, 30
            for k, line in enumerate([text_frame_id, text_timestamp, text_distance, text_caption]):
                y = y0 + k * dy
                (text_w, text_h), _ = cv2.getTextSize(line, font, font_scale, font_thickness)
                # Daha iyi okunabilirlik için metnin arkasına siyah dikdörtgen çiz
                cv2.rectangle(frame, (5, y - text_h - 5), (5 + text_w + 5, y + 5), bg_color, -1)
                cv2.putText(frame, line, (10, y), font, font_scale, text_color, font_thickness, cv2.LINE_AA)

            print(f"\nGörselleştirilen Sonuç {i+1}:")
            print(f"  {text_frame_id}, {text_timestamp}, {text_distance}")
            print(f"  {text_caption}")
            cv2_imshow(frame)
        else:
            print(f"Frame {frame_id} okunamadı.")
else:
    print("Görselleştirilecek arama sonucu bulunamadı.")

cap.release()
print("\n✅ Keyframe görselleştirme tamamlandı.")

Output hidden; open in https://colab.research.google.com to view.